<a href="https://colab.research.google.com/github/BigDanVader/COVID_Scrapper/blob/main/HW5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [33]:
!pip install -U spacy
!pip install newsapi-python
import en_core_web_lg
from newsapi import NewsApiClient
import pickle
import pandas as pd 
from collections import Counter
import token

nlp = en_core_web_lg.load()
newsapi = NewsApiClient (api_key='8586c8f99df3450f82ad10220d27e4aa')

articles = []
for pagina in range(1, 6):
  temp = newsapi.get_everything(q='coronavirus', language='en', from_param='2021-02-24', to='2021-03-23', sort_by='relevancy', page=pagina)
  articles.append(temp)

filename = 'articlesCOVID.pckl'
pickle.dump(articles, open(filename, 'wb'))
filename = 'articlesCOVID.pckl'
loaded_model = pickle.load(open(filename, 'rb'))
filepath = '/articlesCOVID.pckl'
pickle.dump(loaded_model, open(filepath, 'wb'))

dados = []
for i, article in enumerate(articles):
    for x in article['articles']:
        title = x['title']
        date = x['publishedAt']
        description = x['description']
        content = x['content']
        dados.append({'title':title, 'date':date, 'desc':description, 'content':content})
df = pd.DataFrame(dados)
df = df.dropna()
df.head()

def get_keywords_eng(record):
  doc = nlp(content)
  pos_tag = ['NOUN', 'VERB', 'PROPN']
  result = []
  for token in doc:
    if (token.text in nlp.Defaults.stop_words or token.is_punct ):
      continue
    if (token.pos_ in pos_tag):
      result.append(token.text)
  return result

results = []
for content in df.content.values:
    results.append([('#' + x[0]) for x in Counter(get_keywords_eng(content)).most_common(5)])
df['keywords'] = results
df.to_csv(r'/output.csv')

Requirement already up-to-date: spacy in /usr/local/lib/python3.7/dist-packages (3.0.5)
